In [14]:
import sys

python_path = ['/home/claudio/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages/opencv-4.1.0-py3.6.egg/cv2',
               '/home/sebastian/.local/lib/python3.6/site-packages/cv2','/home/sebastian/dat/onnx/onnx-tensorrt/third_party/onnx']
sys.path.extend(python_path)
from torchnlp.word_to_vector import FastText
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
pretrained_embedding = FastText(cache='/data/word_embeddings/')

In [ ]:
pretrained_embedding['']

In [33]:
import pandas as pd
from utils import *
from zero_shot_model import *

In [19]:
df = pd.read_pickle('tags_onehot_corrected.pkl')

In [20]:
df.columns

Index(['ballroom', 'bar_lounge', 'basics', 'bathroom', 'beach', 'breakfast',
       'business_center', 'city_view', 'exterior_view', 'golf_course',
       'guest_room', 'guest_room_amenities', 'health_club', 'hot_news',
       'hotel_front', 'interior_view', 'kitchen', 'living_room', 'lobby_view',
       'logo', 'map', 'meeting_room', 'miscellaneous', 'natural_view',
       'pool_view', 'promotional', 'property_amenities',
       'recreational_facility', 'restaurant', 'spa', 'suite', 'file'],
      dtype='object')

In [45]:
category_to_word = {'ballroom':'ballroom', 'bar_lounge':'bar', 'bathroom':'bathroom', 'beach':'beach', 'breakfast':'breakfast', 'city_view': 'city', 
                   'golf_course':'golfcourse', 'guest_room':'bedroom', 'health_club':'gym', 'hotel_front':'facade', 'kitchen':'kitchen', 'living_room':'livingroom',
                   'lobby_view':'lobby', 'meeting_room':'meetingroom', 'natural_view':'mountain', 'pool_view':'pool', 'recreational_facility':'recreational',
                   'restaurant':'restaurant', 'spa':'spa'}

In [48]:
len(category_to_word.keys())

19

In [ ]:
category_embeddings = {}
for k in category_to_word.keys():
    category_embeddings[k] = pretrained_embedding[category_to_word[k]]

In [59]:
from zero_shot_model import *

In [46]:
device = 'cuda:0'

In [54]:
model = ZeroshotModel(device, load_pretrained_networks=True)

In [38]:
images = torch.randn(10,3,448,448).to(device)

In [53]:
model(images)[0].shape

/home/claudio/.local/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


torch.Size([10, 300])

In [56]:
embedding_matrix = get_category_embedding_matrix()

In [40]:
data = get_zeroshot_dataset(True)

/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  samples = data[data[cat] == 1]


In [6]:
data.columns

Index(['ballroom', 'bar_lounge', 'bathroom', 'beach', 'breakfast', 'city_view',
       'golf_course', 'guest_room', 'health_club', 'living_room', 'lobby_view',
       'meeting_room', 'pool_view', 'recreational_facility', 'spa', 'file'],
      dtype='object')

In [7]:
test_data = get_zeroshot_dataset(False)

/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['file'] = tags['file']


In [45]:
dataset = ZeroshotDataset('/data/hotel_images/', data)

In [28]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

In [29]:
it = iter(dataloader)

In [30]:
images , labels = next(it)

In [31]:
labels.shape

torch.Size([64, 15])

In [32]:
images.shape

torch.Size([64, 3, 448, 448])

In [34]:
get_category_embedding_matrix().shape

torch.Size([300, 19])

In [43]:
def embedding_sofmax_loss(projections, labels, embedding_matrix):
    logits = torch.mm(projections, embedding_matrix)
    return torch.nn.functional.cross_entropy(logits, labels)

In [55]:
optimizer = torch.optim.Adam(model.get_model_parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0, amsgrad=False)

In [57]:
epochs = 5

for epoch in range(epochs):
    
    for images, labels in dataloader:
        
        optimizer.zero_grad()
        
        projections = model(images.to(device))
        
        loss = embedding_sofmax_loss(projections, labels.to(device), embedding_matrix)
        
        loss.backward()
        
        optimizer.step()
        
        model.eval_mode()
        
        with torch.no_grad():
        
            for images, labels in test_data:
                
                projections = model(images.to(device))
                
                logits = torch.mm(projections, embedding_matrix)
                
                pred, indices_pred = torch.topk(1, logits)
                
                indices_true = torch.topk(1, labels.to(device))
                
                print(indices_true.shape)
                print(indices_pred.shape)
            
            
    

/home/claudio/.local/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 11.91 GiB total capacity; 10.59 GiB already allocated; 601.06 MiB free; 10.75 GiB reserved in total by PyTorch)